<a href="https://colab.research.google.com/github/felanbi/Projet-OC-Categoriser-automatiquement-des-questions/blob/main/P5_02_notebooktest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import nltk
import pickle

In [3]:
data = pd.read_pickle("drive/MyDrive/data/data_tokenized_sw_free_v2.pkl")
data.head()

,post,tags
Id,,
65526399,node taking hour employee recordsgroovy xmlbel...,[javascript]
65526401,pythonuser usingibased game game begin create ...,[python]
65526407,remove ver powered whmcomplete solution whmcsi...,"[php, css]"
65526419,object allocated allocate constructremoved des...,[c++]
65526447,react route change createbrowserhistory blockr...,"[javascript, reactjs]"


In [24]:
data.tags.map(lambda x: " ".join(x)).value_counts()[:20]

python                 49102
java                   22906
javascript             19315
c#                     18535
android                14318
c++                    13742
flutter                12166
r                      11971
reactjs                11640
php                     9578
javascript reactjs      8994
python pandas           8800
node.js                 7472
python python-3.x       7217
amazon-web-services     7191
angular                 6282
ios swift               4881
javascript node.js      4773
python-3.x              4445
swift                   4425
Name: tags, dtype: int64

In [31]:
liste = []
for list_tag in data.tags.head(50):
  liste.extend(list_tag)

from collections import Counter
{k : v/len(liste) for k,v in Counter(liste).items()}

{'android': 0.041666666666666664,
 'c#': 0.05555555555555555,
 'c++': 0.027777777777777776,
 'css': 0.05555555555555555,
 'flutter': 0.027777777777777776,
 'html': 0.05555555555555555,
 'ios': 0.08333333333333333,
 'java': 0.09722222222222222,
 'javascript': 0.125,
 'node.js': 0.027777777777777776,
 'pandas': 0.041666666666666664,
 'php': 0.027777777777777776,
 'python': 0.125,
 'python-3.x': 0.027777777777777776,
 'r': 0.013888888888888888,
 'reactjs': 0.06944444444444445,
 'swift': 0.05555555555555555,
 'typescript': 0.041666666666666664}

In [4]:
df = data.head(150000).copy()
df.post = df.post.str.split()
df.head()

,post,tags
Id,,
65526399,"[node, taking, hour, employee, recordsgroovy, ...",[javascript]
65526401,"[pythonuser, usingibased, game, game, begin, c...",[python]
65526407,"[remove, ver, powered, whmcomplete, solution, ...","[php, css]"
65526419,"[object, allocated, allocate, constructremoved...",[c++]
65526447,"[react, route, change, createbrowserhistory, b...","[javascript, reactjs]"


In [5]:
vocabulary = pickle.load(open("drive/MyDrive/data/vocabulary.pkl", "rb"))

In [6]:
from gensim.models import TfidfModel
import gensim.corpora as corpora
from gensim import models
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import matplotlib.pyplot as plt

texts = df.post.to_list()
# id2word = corpora.Dictionary(texts)
# id2word.filter_extremes(keep_n = 10000)

id2word = corpora.Dictionary()
id2word.add_documents([[x] for x in vocabulary])

bow_corpus = [id2word.doc2bow(text) for text in texts]
tfidf = TfidfModel(bow_corpus)
tfidf_corpus = [tfidf[text] for text in bow_corpus]

model = LdaMulticore(corpus = tfidf_corpus,
                    id2word = id2word,
                    num_topics = 20, # nombre de tags retenus
                    random_state = 42,
                    passes = 10,
                    workers = 7,
                    minimum_probability = 0)

coherencemodel = CoherenceModel(model = model, 
                                texts = texts,
                                dictionary = id2word, 
                                coherence = 'c_v')

coherencemodel.get_coherence()

0.5936756795004066

In [7]:
pickle.dump(model, open("drive/MyDrive/data/lda_model.pkl", "wb"))

In [8]:
from pprint import pprint

print('\nPerplexity: ', model.log_perplexity(tfidf_corpus))

model_topics = model.show_topics(formatted=False)
pprint(model.print_topics(num_words=30))


Perplexity:  -6.61389697858361
[(0,
  '0.043*"string" + 0.039*"public" + 0.029*"method" + 0.029*"map" + '
  '0.021*"property" + 0.019*"object" + 0.018*"test" + 0.018*"convert" + '
  '0.017*"private" + 0.015*"system" + 0.015*"list" + 0.014*"key" + '
  '0.013*"null" + 0.013*"int" + 0.012*"result" + 0.011*"static" + 0.011*"json" '
  '+ 0.011*"set" + 0.011*"writeline" + 0.011*"following" + 0.010*"create" + '
  '0.010*"interface" + 0.010*"var" + 0.008*"question" + 0.008*"try" + '
  '0.008*"check" + 0.008*"below" + 0.008*"using" + 0.007*"don" + '
  '0.007*"exception"'),
 (1,
  '0.047*"page" + 0.022*"link" + 0.022*"login" + 0.022*"url" + 0.021*"route" + '
  '0.020*"php" + 0.019*"www" + 0.019*"request" + 0.018*"tab" + 0.017*"content" '
  '+ 0.013*"html" + 0.012*"auth" + 0.011*"tag" + 0.011*"https" + 0.011*"com" + '
  '0.010*"vue" + 0.010*"http" + 0.010*"google" + 0.010*"script" + 0.010*"api" '
  '+ 0.009*"index" + 0.009*"response" + 0.009*"header" + 0.008*"site" + '
  '0.008*"laravel" + 0.008

In [9]:
# original_post = data['post']
# original_cleaned_keywords = data['splitted_tags']

def format_topics_sentences(ldamodel, corpus):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=model, corpus=tfidf_corpus)
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords
0,18.0,0.7719,"org, java, dependency, plugin, spring, jar, gr..."
1,7.0,0.7668,"print, self, def, train, cv, tf, shape, str, m..."
2,1.0,0.8151,"page, link, login, url, route, php, www, reque..."
3,2.0,0.4376,"std, int, vector, float, operator, template, c..."
4,16.0,0.8310,"react, component, const, node, js, div, prop, ..."
...,...,...,...
149995,8.0,0.8251,"google, play, angular, node, device, firebase,..."
149996,3.0,0.5266,"studio, visual, library, microsoft, socket, ex..."
149997,13.0,0.7559,"val, view, fun, activity, intent, layout, dp, ..."
149998,0.0,0.6555,"string, public, method, map, property, object,..."


In [13]:
df_topic_sents_keywords.index = df.index
df_topic_sents_keywords.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords
Id,,,
65526399,18.0,0.7719,"org, java, dependency, plugin, spring, jar, gr..."
65526401,7.0,0.7668,"print, self, def, train, cv, tf, shape, str, m..."
65526407,1.0,0.8151,"page, link, login, url, route, php, www, reque..."
65526419,2.0,0.4376,"std, int, vector, float, operator, template, c..."
65526447,16.0,0.8310,"react, component, const, node, js, div, prop, ..."


In [22]:
df_tmp = df_topic_sents_keywords.copy()
df_tmp.sort_values(by = ["Perc_Contribution"], ascending = False, inplace = True)
df_tmp.drop_duplicates(subset = ["Dominant_Topic"], inplace = True)
df_tmp = df_tmp.join(df[["tags"]])

df_tmp[["Dominant_Topic", "Topic_Keywords", "tags"]]

,Dominant_Topic,Topic_Keywords,tags
Id,,,
60902935,14.0,"const, await, log, async, err, response, conso...","[javascript, node.js]"
66436027,6.0,"child, context, flutter, dart, icon, buildcont...",[flutter]
61124754,1.0,"page, link, login, url, route, php, www, reque...",[javascript]
60090428,16.0,"react, component, const, node, js, div, prop, ...",[reactjs]
67368311,8.0,"google, play, angular, node, device, firebase,...",[flutter]
67807054,5.0,"py, python, install, site, package, run, usr, ...",[python]
59567602,17.0,"aws, access, ec, resource, cluster, permission...","[node.js, amazon-web-services]"
65611164,19.0,"div, px, style, td, span, li, font, item, img,...",[javascript]
60372904,18.0,"org, java, dependency, plugin, spring, jar, gr...",[java]


In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
mlb.fit_transform(df_tmp.tags.to_list())

len(mlb.classes_)

12

In [19]:
lda_model = model
def predict_unsupervised_tags(text):
    """
    Predict tags of a preprocessed text
    
    Args:
        text(list): preprocessed text
        
    Returns:
        relevant_tags(list): list of tags
    """
    
    corpus_new = id2word.doc2bow(text)
    topics = lda_model.get_document_topics(corpus_new)
    
    #find most relevant topic according to probability
    relevant_topic = topics[0][0]
    relevant_topic_prob = topics[0][1]
    
    for i in range(len(topics)):
        if topics[i][1] > relevant_topic_prob:
            relevant_topic = topics[i][0]
            relevant_topic_prob = topics[i][1]
            
    #retrieve associated to topic tags present in submited text
    potential_tags = lda_model.get_topic_terms(topicid=relevant_topic, topn=20)
    
    relevant_tags = [id2word[tag[0]] for tag in potential_tags if id2word[tag[0]] in text]
    
    return relevant_tags

for i in range(20):
    print(f"Liste de tags à prédire : {df.tags.iloc[i]}\n")
    print(f"Liste des tags prédits par le modèle non supervisé: {predict_unsupervised_tags(df.post.iloc[i])}\n\n")

# ajouter les sorties supervisées

Liste de tags à prédire : ['javascript']

Liste des tags prédits par le modèle non supervisé: ['response', 'message', 'function']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['print', 'self', 'def', 'len', 'input']


Liste de tags à prédire : ['php', 'css']

Liste des tags prédits par le modèle non supervisé: ['found', 'version']


Liste de tags à prédire : ['c++']

Liste des tags prédits par le modèle non supervisé: ['std']


Liste de tags à prédire : ['javascript', 'reactjs']

Liste des tags prédits par le modèle non supervisé: ['react', 'const', 'prop', 'useeffect']


Liste de tags à prédire : ['java']

Liste des tags prédits par le modèle non supervisé: ['string', 'public', 'private', 'int', 'set', 'following']


Liste de tags à prédire : ['python']

Liste des tags prédits par le modèle non supervisé: ['extension']


Liste de tags à prédire : ['ios', 'flutter']

Liste des tags prédits par le modèle non supervisé: ['firebase', 'dev', '

In [ ]:
# retirer les tfidf faibles
# les afficher